In [1]:
import numpy as np
import xtrack as xt
import xdeps as xd

from cpymad.madx import Madx

fname = ".lattice_files/fccee_t_V22.seq"
pc_gev = 182.5

mad = Madx()
mad.call(fname)
mad.beam(particle='positron', pc=pc_gev)
mad.use('fccee_p_ring')

mad.call('./install_wigglers.madx')
mad.input("exec, define_wigglers_as_multipoles()")
mad.input("exec, install_wigglers()")
mad.use('fccee_p_ring')

line_thick = xt.Line.from_madx_sequence(mad.sequence.fccee_p_ring, allow_thick=True,
                                  deferred_expressions=True)




  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.09.03  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2024.04.25               +
  + Execution date: 2024.12.09 16:46:55      +
  ++++++++++++++++++++++++++++++++++++++++++++
!

!SAD2MADX V0.4.1 (11/5/2018)

!Conversion of FCCee_t_529_nosol.sad at Fri Dec 10 08:50:01 JST 2021

!

AXISAD =3.0185567860467553e-12; BXISAD =1.0000000000017404; AYISAD =5.952306930154074e-13; BYISAD =.0016000000000024985; EXISAD =1.4742338456741096e-14; EPXISAD =8.584873374143616e-14; EYISAD =5.842301466747665e-19; EPYISAD =-3.414776091098357e-16;

K1QC1L1 = -.16097717432405162;

LQC1L1 = .7;

K1QC1R2 = -.16409289579016573;

LQC1R2 = 1.25;

K1QC1R3 = -.14551404073944188;

LQC1R3 = 1.25;

K1QC2R1 = .0449384431977077;

LQC2R1 = 1.25;

K1QC2R2 = .15845771736262967;

LQC2R2 = 1.25;

K1QT1 = .007256447615584834;

LQT1 = 1;

K1QC3 = -.021506415099589872;

LQC3 = 3.5;

K1QC4 = .014852668903067582;

LQC4 = 3.5;

Converting sequence "fccee_p_ring": 100%|██████████| 17950/17950 [00:36<00:00, 495.65it/s]


In [11]:
# Introduce 90 degree tilt for wiggler
tt = line_thick.get_table()
print(len(tt.rows["mwi.*"]))
print(len(tt.element_type=="Bend"))
wigs = tt.rows['mwi.*'].name
# print(wigs)
for nn in wigs:
    line_thick.element_refs[nn].rot_s_rad = np.pi/2


line_thick.to_json(".lattice_files/fccee_t_V22_thick_with_wigglers.json")


72
17951


In [12]:
line = line_thick.copy()
Strategy = xt.slicing.Strategy
Teapot = xt.slicing.Teapot
slicing_strategies = [
    Strategy(slicing=Teapot(1)),  # Default catch-all as in MAD-X
    Strategy(slicing=Teapot(3), element_type=xt.Bend),
    Strategy(slicing=Teapot(3), element_type=xt.CombinedFunctionMagnet),
    # Strategy(slicing=Teapot(50), element_type=xt.Quadrupole), # Starting point
    Strategy(slicing=Teapot(5), name=r'^qf.*'),
    Strategy(slicing=Teapot(5), name=r'^qd.*'),
    Strategy(slicing=Teapot(5), name=r'^qfg.*'),
    Strategy(slicing=Teapot(5), name=r'^qdg.*'),
    Strategy(slicing=Teapot(5), name=r'^ql.*'),
    Strategy(slicing=Teapot(5), name=r'^qs.*'),
    Strategy(slicing=Teapot(10), name=r'^qb.*'),
    Strategy(slicing=Teapot(10), name=r'^qg.*'),
    Strategy(slicing=Teapot(10), name=r'^qh.*'),
    Strategy(slicing=Teapot(10), name=r'^qi.*'),
    Strategy(slicing=Teapot(10), name=r'^qr.*'),
    Strategy(slicing=Teapot(10), name=r'^qu.*'),
    Strategy(slicing=Teapot(10), name=r'^qy.*'),
    Strategy(slicing=Teapot(50), name=r'^qa.*'),
    Strategy(slicing=Teapot(50), name=r'^qc.*'),
    Strategy(slicing=Teapot(20), name=r'^sy\..*'),
    Strategy(slicing=Teapot(30), name=r'^mwi\..*'),
]

line.slice_thick_elements(slicing_strategies=slicing_strategies)
line.to_json(".lattice_files/fccee_t_V22_thin_with_wigglers.json")


Slicing line: 100%|██████████| 17950/17950 [00:03<00:00, 4999.96it/s]
